In [1]:
import pandas as pd

In [2]:
dfs = []

In [3]:
for project in ['cinder', 'glance', 'neutron', 'nova']:
    df = pd.read_csv(f'../data/rq1/rq1_{project}.csv')
    df['project'] = project
    dfs.append(df)

In [4]:
df = pd.concat(dfs, ignore_index=True)

In [5]:
len(df)

204796

In [6]:
df['author_date'] = df['author_date'].apply(
    lambda x : x[:10]
)
df['commit_date'] = df['commit_date'].apply(
    lambda x : x[:10]
)

In [7]:
df.to_csv('../data/rq1/rq1_all.csv', index=False)